In [2]:
!pip install -q torch transformers peft trl datasets accelerate bitsandbytes langchain sentence-transformers  accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━

In [3]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
!locale.getpreferredencoding = getpreferredencoding

/bin/bash: line 1: locale.getpreferredencoding: command not found


In [19]:
!%reload_ext dotenv
!%dotenv

import os
import warnings
import torch
import pandas as pd
from tqdm.notebook import tqdm
from datasets import Dataset,load_dataset,DatasetDict



from tqdm import tqdm
from accelerate import Accelerator
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    TaskType,
    get_peft_model,
    prepare_model_for_int8_training,
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    default_data_collator,
    BitsAndBytesConfig,
    get_linear_schedule_with_warmup
)
from dataclasses import dataclass, field
from torch.utils.data import Dataset, DataLoader
from enum import Enum
from datasets import load_dataset,DatasetDict
from typing import Union , Dict,Optional,Any,List

/bin/bash: line 1: fg: no job control
/bin/bash: line 1: fg: no job control


In [5]:



def Advanced_Data_Loader(input: Union[str, Dict[str, str]], format: Optional[str] = None, split_ratios: Optional[Dict[str, float]] = None) -> Optional[DatasetDict]:
    """
    Loads a dataset from a given input path or dictionary specifying file paths and splits it.

    :param input: A string representing the dataset name or directory, or a dictionary containing file paths.
    :param format: The format of the dataset if loading from a file (e.g., 'csv' or 'json').
    :param split_ratios: A dictionary with keys 'train', 'test', and 'eval' containing split ratios.
    :return: A loaded and split dataset or None in case of failure.
    """
    if split_ratios is None:
        split_ratios = {'train': 0.8, 'test': 0.1, 'eval': 0.1}

    try:
        # Load the dataset
        if isinstance(input, dict) and format in ['csv', 'json','jsonl']:
            dataset = load_dataset(format, data_files=input)
        elif isinstance(input, str) and format == 'text':
            dataset = load_dataset(format, data_dir=input)
        elif isinstance(input, str) and format is None:
            dataset = load_dataset(input)
        else:
            warnings.warn("Invalid input or format. Please provide a valid dataset name, directory, or file paths.")
            return None
    except FileNotFoundError as e:
        warnings.warn(str(e))
        return None

    # Split the dataset
    if dataset:
        split_dataset = dataset['train'].train_test_split(test_size=split_ratios['test'] + split_ratios['eval'])
        test_eval_dataset = split_dataset['test'].train_test_split(test_size=split_ratios['eval'] / (split_ratios['test'] + split_ratios['eval']))
        dataset = DatasetDict({
            'train': split_dataset['train'],
            'test': test_eval_dataset['train'],
            'eval': test_eval_dataset['test']
        })

    print("Splits: ", dataset.keys())
    print("Columns: ", {split: dataset[split].column_names for split in dataset.keys()})
    return dataset

In [6]:
def Create_Tokenizer(tokenizer_name_or_path: str = 'gpt2') -> AutoTokenizer:
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id
    if tokenizer.bos_token_id is None:
        tokenizer.bos_token_id = tokenizer.pad_token_id
    if tokenizer.eos_token_id is None:
        tokenizer.eos_token_id = tokenizer.pad_token_id
    if tokenizer.unk_token_id is None:
        tokenizer.unk_token_id = tokenizer.pad_token_id
    if tokenizer.sep_token_id is None:
        tokenizer.sep_token_id = tokenizer.pad_token_id
    if tokenizer.cls_token_id is None:
        tokenizer.cls_token_id = tokenizer.pad_token_id
    if tokenizer.mask_token_id is None:
        tokenizer.mask_token_id = tokenizer.pad_token_id
    return tokenizer

In [7]:
def Create_Model(Model_Name_or_path:str='gpt2')->AutoModelForCausalLM:



  model=AutoModelForCausalLM.from_pretrained(Model_Name_or_path)
  return model



In [ ]:
!rm -rf /content/sample_data

In [ ]:
!git clone https://huggingface.co/datasets/nvidia/OpenMathInstruct-1.git

Cloning into 'OpenMathInstruct-1'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 77 (delta 21), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (77/77), 508.52 KiB | 3.10 MiB/s, done.
Filtering content: 100% (4/4), 4.32 GiB | 20.51 MiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	incorrect_solutions/train.jsonl

See: `git lfs help smudge` for more details.


In [ ]:
!rm -rf

In [8]:
dataset=Advanced_Data_Loader(input='fka/awesome-chatgpt-prompts')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Splits:  dict_keys(['train', 'test', 'eval'])
Columns:  {'train': ['act', 'prompt'], 'test': ['act', 'prompt'], 'eval': ['act', 'prompt']}


In [ ]:
train_data=dataset['train']

In [9]:
Columns=dataset['train'].column_names
print(Columns)

['act', 'prompt']


In [35]:
TEXT_COLUMN='act'
LABEL_COLUMN='prompt'
MAX_LENGTH=512
BATCH_SIZE=1

In [ ]:
dataset=Advanced_Data_Loader('/content/OpenMathInstruct-1')

Splits:  dict_keys(['train', 'test', 'eval'])
Columns:  {'train': ['question', 'expected_answer', 'predicted_answer', 'error_message', 'is_correct', 'generation_type', 'dataset', 'generated_solution'], 'test': ['question', 'expected_answer', 'predicted_answer', 'error_message', 'is_correct', 'generation_type', 'dataset', 'generated_solution'], 'eval': ['question', 'expected_answer', 'predicted_answer', 'error_message', 'is_correct', 'generation_type', 'dataset', 'generated_solution']}


In [36]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)
tokenizer=Create_Tokenizer(tokenizer_name_or_path="gpt2")
model=Create_Model("gpt2")
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 126,799,104 || trainable%: 1.8606566809809635


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:711: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [37]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
        

In [14]:
def Preprocess_Function(examples):
  """
  Preprocess the dataset.
  """

  batch_size = len(examples[TEXT_COLUMN])
  inputs = [f"{TEXT_COLUMN } : {x} Label : " for x in examples[TEXT_COLUMN ]]
  targets = [str(x) for x in examples[LABEL_COLUMN]]
  model_inputs = tokenizer(inputs)
  labels = tokenizer(targets, add_special_tokens=False)  # don't add bos token because we concatenate with inputs
  for i in range(batch_size):
      sample_input_ids = model_inputs["input_ids"][i]
      label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]
      model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
      labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
      model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
  for i in range(batch_size):
      sample_input_ids = model_inputs["input_ids"][i]
      label_input_ids = labels["input_ids"][i]
      model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
          MAX_LENGTH - len(sample_input_ids)
      ) + sample_input_ids
      model_inputs["attention_mask"][i] = [0] * (MAX_LENGTH - len(sample_input_ids)) + model_inputs[
          "attention_mask"
      ][i]
      labels["input_ids"][i] = [-100] * (MAX_LENGTH - len(sample_input_ids)) + label_input_ids
      model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:MAX_LENGTH])
      model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:MAX_LENGTH])
      labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:MAX_LENGTH])
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [38]:
accelerator = Accelerator()

with accelerator.main_process_first():

  processed_datasets = dataset.map(
            Preprocess_Function,
            batched=True,
            num_proc=1,
            remove_columns=dataset["train"].column_names,
            load_from_cache_file=True,
            desc="""
            CDAC-B fine-tuning based on indian language
            NOTE: humanity benefit's
            """,
)


            CDAC-B fine-tuning based on indian language
            NOTE: humanity benefit's
            :   …


            CDAC-B fine-tuning based on indian language
            NOTE: humanity benefit's
            :   …


            CDAC-B fine-tuning based on indian language
            NOTE: humanity benefit's
            :   …

In [16]:
train_dataset=processed_datasets['train']
test_dataset=processed_datasets['test']
eval_dataset=processed_datasets['eval']

In [17]:
train_dataset=processed_datasets['train']
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=BATCH_SIZE, pin_memory=True
)

In [18]:
# optimizer and lr scheduler
lr=0.0001
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

In [20]:
num_epochs=10
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
torch.cuda.reset_peak_memory_stats()
torch.cuda.memory_cached()

In [ ]:
torch.cuda.memory_stats()

In [ ]:
del tensors, output

## Here are some commands related to emptying the GPU cache in PyTorch:

1. `torch.cuda.empty_cache()`: Empties the GPU cache and frees up GPU memory.
2. `torch.cuda.reset_peak_memory_stats()`: Resets the peak memory usage statistics for the current GPU.
3. `torch.cuda.memory_allocated()`: Returns the total amount of GPU memory currently allocated.
4. `torch.cuda.memory_cached()`: Returns the total amount of GPU memory currently cached.
5. `torch.cuda.memory_reserved()`: Returns the total amount of GPU memory currently reserved.
6. `torch.cuda.max_memory_allocated()`: Returns the maximum amount of GPU memory allocated during the lifetime of the current process.
7. `torch.cuda.memory_summary(device=None)`: Displays GPU memory usage statistics for the specified device.
8. `torch.cuda.memory_stats()`: Returns a dictionary of GPU memory usage statistics.
9. `torch.cuda.synchronize()`: Blocks the CPU until all pending asynchronous GPU operations are completed.
10. `del tensors`: Deletes tensors to free up GPU memory.
11. `gc.collect()`: Runs the garbage collector to free up memory.
12. `torch.Tensor.cpu()`: Moves a tensor from GPU memory to CPU memory.
13. `torch.Tensor.to('cpu')`: Moves a tensor from GPU memory to CPU memory.
14. `torch.Tensor.cuda()`: Moves a tensor from CPU memory to GPU memory.
15. `torch.Tensor.to('cuda')`: Moves a tensor from CPU memory to GPU memory.
16. `torch.cuda.set_device(device)`: Sets the current GPU device to be used for subsequent operations.
17. `torch.device('cuda:0')` or `torch.device('cuda')` or `torch.device('cuda:<device_index>')`: Specifies the GPU device to be used for subsequent operations.
18. `torch.nn.DataParallel(model, device_ids=[0,1,2])`: Wraps a model with DataParallel to replicate it across multiple GPUs.
19. `torch.nn.parallel.DistributedDataParallel(model, device_ids=[0,1,2])`: Wraps a model with DistributedDataParallel to replicate it across multiple GPUs and multiple machines.
20. `torch.backends.cudnn.benchmark = True`: Enables cuDNN benchmark mode to automatically choose the best algorithm for the current hardware.
21. `torch.backends.cudnn.deterministic = True`: Enables cuDNN deterministic mode to ensure reproducible results.
22. `torch.backends.cudnn.enabled = False`: Disables cuDNN to use generic GPU operations instead.
23. `torch.autograd.set_grad_enabled(False)`: Disables gradient computation for subsequent operations.
24. `torch.no_grad()`: Context manager to disable gradient computation for subsequent operations.
25. `torch.utils.dlpack.to_dlpack()`: Converts a PyTorch tensor to DLPack format to share GPU memory with other frameworks.


In [72]:
torch.cuda.empty_cache()

In [71]:
torch.cuda.memory_summary()

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 8            |        cudaMalloc retries: 8         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  14693 MiB |  14885 MiB |  74519 MiB |  59825 MiB |\n|       from large pool |  14623 MiB |  14815 MiB |  74370 MiB |  59747 MiB |\n|       from small pool |     70 MiB |     71 MiB |    148 MiB |     77 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |  14693 MiB |  14885 MiB |  74519 MiB |  59825 MiB |\n|       from large pool |  14623 MiB |  14815 MiB |

In [48]:
!nvidia-smi -q -d Memory | grep -A4 GPU

Attached GPUs                             : 1
GPU 00000000:00:04.0
    FB Memory Usage
        Total                             : 15360 MiB
        Reserved                          : 257 MiB
        Used                              : 5681 MiB


In [49]:
!nvidia-smi -q -d Memory


==============NVSMI LOG==============

Timestamp                                 : Mon Feb 19 09:04:41 2024
Driver Version                            : 535.104.05
CUDA Version                              : 12.2

Attached GPUs                             : 1
GPU 00000000:00:04.0
    FB Memory Usage
        Total                             : 15360 MiB
        Reserved                          : 257 MiB
        Used                              : 5681 MiB
        Free                              : 9421 MiB
    BAR1 Memory Usage
        Total                             : 256 MiB
        Used                              : 5 MiB
        Free                              : 251 MiB
    Conf Compute Protected Memory Usage
        Total                             : 0 MiB
        Used                              : 0 MiB
        Free                              : 0 MiB



In [67]:
!nvidia-smi  --list-gpus

GPU 0: Tesla T4 (UUID: GPU-f0a12b99-8489-7e02-f718-53b480486e6d)


#  `nvidia-smi` (NVIDIA System Management Interface):

1. `nvidia-smi`: Display the current GPU usage and other information.
2. `nvidia-smi -q`: Display detailed information about the GPUs in a machine-readable format.
3. `nvidia-smi -l`: Display the GPU utilization log.
4. `nvidia-smi -i <GPU ID>`: Display information about a specific GPU.
5. `nvidia-smi -pm ENABLED`: Enable persistence mode, which keeps the GPU drivers loaded even when no GPU processes are running.
6. `nvidia-smi -pm DISABLED`: Disable persistence mode.
7. `nvidia-smi -ac <GPU ID>`: Set the GPU to run in compute mode.
8. `nvidia-smi -ar <GPU ID>`: Set the GPU to run in graphics mode.
9. `nvidia-smi --format=csv -lms 10`: Display the GPU utilization log in CSV format with a 10-second interval.
10. `nvidia-smi --loop-ms=1000`: Continuously display GPU usage information with a 1-second interval.
11. `nvidia-smi --idle-timeout=0`: Prevent the GPU from idling and consuming less power.
12. `nvidia-smi --fan-speed=50`: Set the GPU fan speed to 50%.
13. `nvidia-smi --temperature-threshold=85`: Set the GPU temperature threshold to 85 degrees Celsius.
14. `nvidia-smi --mem-used`: Display the amount of GPU memory used.
15. `nvidia-smi --utilization-gpu`: Display the GPU utilization.
16. `nvidia-smi --utilization-memory`: Display the memory utilization.
17. `nvidia-smi --utilization-encoder`: Display the video encoder utilization.
18. `nvidia-smi --utilization-decoder`: Display the video decoder utilization.
19. `nvidia-smi --power-usage`: Display the GPU power usage.
20. `nvidia-smi --power-limit`: Display the GPU power limit.
21. `nvidia-smi --power-limit-max`: Set the GPU power limit to its maximum value.
22. `nvidia-smi --power-limit-min`: Set the GPU power limit to its minimum value.
23. `nvidia-smi --applications`: Display information about the GPU applications currently running.
24. `nvidia-smi --processes`: Display information about the GPU processes currently running.
25. `nvidia-smi --help`: Display help information for the `nvidia-smi` command.
26. `nvidia-smi -L`: Display information about the GPUs in the system, including their names and PCI bus IDs.
27. `nvidia-smi -d MEMORY`: Display detailed information about the GPU memory usage.
28. `nvidia-smi -d UTILIZATION`: Display detailed information about the GPU utilization.
29. `nvidia-smi -d TEMPERATURE`: Display detailed information about the GPU temperature.
30. `nvidia-smi -d POWER`: Display detailed information about the GPU power usage.
31. `nvidia-smi --watch-mem-usage=1`: Continuously display the GPU memory usage with a 1-second interval.
32. `nvidia-smi --watch-utilization=1`: Continuously display the GPU utilization with a 1-second interval.
33. `nvidia-smi --watch-temperature=1`: Continuously display the GPU temperature with a 1-second interval.
34. `nvidia-smi --watch-power=1`: Continuously display the GPU power usage with a 1-second interval.
35. `nvidia-smi --query-gpu=index,name,driver_version,memory.total,memory.used,memory.free`: Display specific information about the GPUs in a custom format.
36. `nvidia-smi --query-compute-apps=pid,process_name,used_memory`: Display information about the GPU compute applications currently running.
37. `nvidia-smi --query-compute-apps=pid | xargs -I{} kill -9 {}`: Kill all GPU compute applications.
38. `nvidia-smi --query-compute-apps=pid | grep <process name>`: Display information about a specific GPU compute application by name.
39. `nvidia-smi --query-gpu=index | xargs -I{} nvidia-smi -i {} --query-gpu=utilization.gpu`: Display the GPU utilization for each GPU in the system.
40. `nvidia-smi --query-gpu=index | xargs -I{} nvidia-smi -i {} --query-gpu=utilization.memory`: Display the memory utilization for each GPU in the system.
41. `nvidia-smi --query-gpu=index | xargs -I{} nvidia-smi -i {} --query-gpu=power.draw`: Display the power draw for each GPU in the system.
42. `nvidia-smi --query-gpu=index | xargs -I{} nvidia-smi -i {} --query-gpu=temperature.gpu`: Display the GPU temperature for each GPU in the system.
43. `nvidia-smi --query-gpu=index | xargs -I{} nvidia-smi -i {} --query-gpu=memory.total`: Display the total GPU memory for each GPU in the system.
44. `nvidia-smi --query-gpu=index | xargs -I{} nvidia-smi -i {} --query-gpu=memory.used`: Display the used GPU memory for each GPU in the system.
45. `nvidia-smi --query-gpu=index | xargs -I{} nvidia-smi -i {} --query-gpu=memory.free`: Display the free GPU memory for each GPU in the system.
46. `nvidia-smi --query-gpu=index | xargs -I{} nvidia-smi -i {} --query-gpu=utilization.memory --format=csv`: Display the memory utilization for each GPU in CSV format.
47. `nvidia-smi --query-gpu=index | xargs -I{} nvidia-smi -i {} --query-gpu=utilization.gpu --format=csv`: Display the GPU utilization for each GPU in CSV format.
48. `nvidia-smi --query-gpu=index | xargs -I{} nvidia-smi -i {} --query-gpu=power.draw --format=csv`: Display the power draw for each GPU in CSV format.
49. `nvidia-smi --query-gpu=index | xargs -I{} nvidia-smi -i {} --query-gpu=temperature.gpu --format=csv`: Display the GPU temperature for each GPU in CSV format.
50. `nvidia-smi --query-gpu=index | xargs -I{} nvidia-smi -i {} --query-gpu=memory.total,memory.used,memory.free --format=csv`: Display the total, used, and free GPU memory for each GPU in CSV format.


# GPU MEMORY  COMMAND :

1. `nvidia-smi --mem-used`: Display the amount of GPU memory used.
2. `nvidia-smi --mem-free`: Display the amount of GPU memory free.
3. `nvidia-smi --mem-total`: Display the total amount of GPU memory.
4. `nvidia-smi --mem-util`: Display the GPU memory utilization percentage.
5. `nvidia-smi --mem-shared`: Display the amount of GPU memory shared with the system.
6. `nvidia-smi --mem-managed`: Display the amount of GPU memory managed by the NVIDIA driver.
7. `nvidia-smi --mem-reserved`: Display the amount of GPU memory reserved for the operating system.
8. `nvidia-smi --mem-mapped`: Display the amount of GPU memory mapped by the operating system.
9. `nvidia-smi --mem-allocated`: Display the amount of GPU memory allocated by applications.
10. `nvidia-smi --mem-free-and-allocated`: Display the amount of GPU memory free and allocated by applications.
11. `nvidia-smi --mem-free-and-reserved`: Display the amount of GPU memory free and reserved by the operating system.
12. `nvidia-smi --mem-free-and-managed`: Display the amount of GPU memory free and managed by the NVIDIA driver.
13. `nvidia-smi --mem-free-and-shared`: Display the amount of GPU memory free and shared with the system.
14. `nvidia-smi --mem-encrypted`: Display the amount of GPU memory encrypted.
15. `nvidia-smi --mem-async`: Display the amount of GPU memory used for asynchronous transfers.
16. `nvidia-smi --mem-pool-used`: Display the amount of GPU memory used by memory pools.
17. `nvidia-smi --mem-pool-free`: Display the amount of GPU memory free for memory pools.
18. `nvidia-smi --mem-pool-total`: Display the total amount of GPU memory available for memory pools.
19. `nvidia-smi --mem-pool-util`: Display the GPU memory utilization percentage for memory pools.
20. `nvidia-smi --mem-managed-used`: Display the amount of GPU memory managed by the NVIDIA driver that is used.
21. `nvidia-smi --mem-managed-free`: Display the amount of GPU memory managed by the NVIDIA driver that is free.
22. `nvidia-smi --mem-encoder-used`: Display the amount of GPU memory used by video encoders.
23. `nvidia-smi --mem-decoder-used`: Display the amount of GPU memory used by video decoders.
24. `nvidia-smi --mem-curc-used`: Display the amount of GPU memory used by the CUDA Runtime Compilation cache.
25. `nvidia-smi --mem-curc-size`: Display the size of the CUDA Runtime Compilation cache.

In [62]:
!nvidia -q -l

/bin/bash: line 1: nvidia: command not found


In [59]:
!nvidia-smi -q -d Memory


==============NVSMI LOG==============

Timestamp                                 : Mon Feb 19 09:16:46 2024
Driver Version                            : 535.104.05
CUDA Version                              : 12.2

Attached GPUs                             : 1
GPU 00000000:00:04.0
    FB Memory Usage
        Total                             : 15360 MiB
        Reserved                          : 257 MiB
        Used                              : 5681 MiB
        Free                              : 9421 MiB
    BAR1 Memory Usage
        Total                             : 256 MiB
        Used                              : 5 MiB
        Free                              : 251 MiB
    Conf Compute Protected Memory Usage
        Total                             : 0 MiB
        Used                              : 0 MiB
        Free                              : 0 MiB



In [ ]:
# training and evaluation
device='cuda'
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()